In [1]:
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.functional import one_hot
from torch import nn, optim
from torch.optim.swa_utils import AveragedModel
from torch.cuda.amp import GradScaler
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms, datasets
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.utils.multiclass import unique_labels
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import networkx as nx
from bs4 import BeautifulSoup
from collections import defaultdict
from pyvis.network import Network
from tabulate import tabulate
from pathlib import Path
import contextlib
import argparse
import sys
import csv

# Local Imports
from musicmap_graph_creator import create_musicmap_graph, compute_shortest_paths_between_classes_nx
from model_resnet18flex import ResNet18Gray
from musicmap_helpers import compute_image_dataset_stats, check_set_gpu, batch_soft_labeling_loss, top_k_accuracy, evaluate, plot_metrics, plot_cm_super, get_input_height


# Base directories for train and val
train_data_directory1 = "./15_second_features_augmented/musicmap_processed_output_splits_train/mel_pcen_gray"
train_data_directory2 = "./15_second_features_augmented/musicmap_processed_output_splits_train/mfcc_gray"
train_data_directory3 = "./15_second_features_augmented/musicmap_processed_output_splits_train/chroma_cq_gray"


# Compute means and SDs for each directory
image_dir1 = train_data_directory1
means1, sds1 = compute_image_dataset_stats(image_dir1)

image_dir2 = train_data_directory2
means2, sds2 = compute_image_dataset_stats(image_dir2)

image_dir3 = train_data_directory3
means3, sds3 = compute_image_dataset_stats(image_dir3)

# Create a csv that records each of the three directories, and the means and sds for each directory
means_sds_df = pd.DataFrame({
    'directory': [image_dir1, image_dir2, image_dir3],
    'mean': [means1, means2, means3],
    'sd': [sds1, sds2, sds3]
})
means_sds_df.to_csv('means_sds.csv', index=False)